In [1]:
# The model definition provided
import sklearn as sk
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import math
import keras as k
from keras import backend as K
# from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Conv1D, Input, Dense, concatenate, Flatten, Add, Activation, BatchNormalization, Reshape
from keras import Model
import tensorflow as tf

Using TensorFlow backend.
/home/h/houruomu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/h/houruomu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/h/houruomu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/h/houruomu/.local/lib/python3.6/site-packages/t

In [2]:
import pandas as pd
import numpy as np

data1 = pd.read_csv("GSE2034-Normal-train.txt", sep="\t")
data2 = pd.read_csv("GSE2034-Tumor-train.txt", sep="\t")
# data format:
# ----------- HEADER LINE ------------
# 2 s1g1 s2g1 s3g1 ... sng1
# 9 s1g2 s2ge s3g2 ... sng2
#          ...........
# k s1g12634 s2g12634 ... sng12634
# use data1.head() to see
data1 = data1.values[:,1:].transpose()
data2 = data2.values[:,1:].transpose()
data1Label = np.zeros(len(data1)).reshape((-1, 1))
data2Label = np.ones(len(data2)).reshape((-1, 1))
x = np.concatenate((data1, data2))
y = np.concatenate((data1Label, data2Label))

In [5]:
dim = 12634
def res1d(x1, layers, kernel=(3,), strides=1,):    
    ConvLayer = k.layers.Conv1D        
    normalizer = BatchNormalization

    c1 = x1        
    c1 = ConvLayer(layers, kernel_size=kernel, use_bias=0, strides=strides, padding='same')(c1)        
    c1 = normalizer(gamma_initializer='zeros')(c1)
    
    o1 = x1
    if strides > 1 or K.int_shape(x1)[-1] != layers:
        if strides > 1:
            o1 = k.layers.ZeroPadding1D((0, strides-1))(o1)
            o1 = k.layers.AveragePooling1D(strides)(o1)
        o1 = ConvLayer(layers, kernel_size=1, use_bias=0, padding='same')(o1)
        o1 = normalizer()(o1)

    v1 = Add()([c1, o1])
#     v1 = K.print_tensor(v1, message="debug")
    v1 = Activation('relu')(v1)
    return v1

def build():
    inp = Input(shape=(dim,)) # length of input
    
    v1 = k.layers.Reshape((-1, 1))(inp)
    v1 = res1d(v1, 4, 3)

    model = k.models.Model(inputs=inp, outputs=v1)
    return model

In [6]:
model = build()


In [7]:
model.compile(optimizer=k.optimizers.Adam(lr=0.0001), metrics=['acc'], loss='binary_crossentropy')
model.predict(data1)

array([[[0.        , 0.        , 0.        , 0.79088026],
        [0.        , 0.        , 0.        , 0.9241283 ],
        [1.0362788 , 0.79011816, 1.1310408 , 0.        ],
        ...,
        [0.5068423 , 0.3864455 , 0.5531903 , 0.        ],
        [0.        , 0.        , 0.        , 0.41297406],
        [0.06686865, 0.05098448, 0.07298343, 0.        ]],

       [[0.        , 0.        , 0.        , 0.99642897],
        [0.        , 0.        , 0.        , 0.53948694],
        [1.6861612 , 1.2856256 , 1.8403516 , 0.        ],
        ...,
        [1.4361778 , 1.095024  , 1.5675087 , 0.        ],
        [0.        , 0.        , 0.        , 0.30074403],
        [0.        , 0.        , 0.        , 0.01920642]],

       [[0.        , 0.        , 0.        , 0.79586226],
        [0.        , 0.        , 0.        , 1.2229016 ],
        [1.4840771 , 1.1315452 , 1.619788  , 0.        ],
        ...,
        [0.72284216, 0.55113614, 0.7889422 , 0.        ],
        [0.        , 0.      